# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:**
Based on the paper titled "Show and Tell: A Neural Image Caption Generator" there is Encoder - Decoder Architecture presented. Encoder is alreay written in model.py so that I added Decoder implementation using LSTM since LSTM performs bettern than RNN in general. As for hyperparameters embeddings is recommended around 50~200  but not over 500 in practice, and I chose 256 for embedding size, 512 for hidden size and 128 as batch size which were tried 32, 64. as all of them fit well in the memory. 

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
Same steps I used in the previous project to apply pre-trained network 
- resize 256 
- downsample the image more than 256x256 using RandomCrop
- RandomHorizontalFlip and Normalize for data augmentation and avoiding overfiting. 
As for batch_size i tried 32,64, 128 based on the recommended range.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 
Pre-trained model which were based on ImageNet has been worked for various tasks even for skin cancer detection since pre-trained model is especially good for any Image Classification task. All parameters are chosen based on previous projects which was also using Resnet-50.
As for Decoder (LSTM), I tried various combintation of traning parameters and model parameters during training Training parameters epochs were only tried 1 and 2 since 3 takes too long time (more than 3 hours)so that I stopped, and  tried embeddings (256,512) batch_size (64, 128) The more batch_size can reduce training time a little bit, however even in 128 I got memory warning when I raised to sequence_size=100. Bigger sequence_size made the loss decreased significantly than other parameters after fixing embedding/hidden = 256. Which means the netmork loosk over 100 sequence and detecting the pattern gets more complicated. As for sequence_size I tried 5, 10, 20, 55, 100. Still hit the loss=3.5 a few times but not converged to loss = 3.5 yet. 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 
Adam optimizer I used to optimize all encoder parameters and decoder parameters together. 

In [5]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128         # batch size
vocab_threshold = 4        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 256          # number of features in hidden state of the RNN decoder
num_epochs = 2             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




          
 12%|█▏        | 50280/414113 [00:26<03:08, 1928.89it/s]


  0%|          | 0/414113 [00:00<?, ?it/s]


  0%|          | 465/414113 [00:00<01:36, 4267.81it/s]


  0%|          | 933/414113 [00:00<01:34, 4381.70it/s]


  0%|          | 1410/414113 [00:00<01:31, 4490.60it/s]


  0%|          | 1889/414113 [00:00<01:30, 4576.07it/s]


  1%|          | 2373/414113 [00:00<01:28, 4651.72it/s]


  1%|          | 2854/414113 [00:00<01:27, 4697.80it/s]


  1%|          | 3323/414113 [00:00<01:27, 4693.58it/s]


  1%|          | 3804/414113 [00:00<01:26, 4726.90it/s]


  1%|          | 4291/414113 [00:00<01:25, 4767.64it/s]


  1%|          | 4772/414113 [00:01<01:25, 4779.87it/s]


  1%|▏         | 5257/414113 [00:01<01:25, 4800.20it/s]


  1%|▏         | 5729/414113 [00:01<01:29, 4564.36it/s]


  1%|▏         | 6203/414113 [00:01<01:28, 4612.89it/s]


  2%|▏         | 6666/414113 [00:01<01:28, 4616.54it/s]


  2%|▏         | 7128/414113 [00:01<01:28, 4614.73it/s]


  2%|▏         

 14%|█▍        | 57056/414113 [00:12<01:15, 4746.57it/s]


 14%|█▍        | 57535/414113 [00:12<01:14, 4757.66it/s]


 14%|█▍        | 58019/414113 [00:12<01:14, 4780.92it/s]


 14%|█▍        | 58498/414113 [00:12<01:14, 4760.05it/s]


 14%|█▍        | 58978/414113 [00:12<01:14, 4769.01it/s]


 14%|█▍        | 59455/414113 [00:12<01:14, 4761.08it/s]


 14%|█▍        | 59937/414113 [00:12<01:14, 4775.57it/s]


 15%|█▍        | 60415/414113 [00:13<01:14, 4738.96it/s]


 15%|█▍        | 60889/414113 [00:13<01:14, 4710.90it/s]


 15%|█▍        | 61362/414113 [00:13<01:14, 4715.38it/s]


 15%|█▍        | 61834/414113 [00:13<01:15, 4690.73it/s]


 15%|█▌        | 62304/414113 [00:13<01:15, 4680.06it/s]


 15%|█▌        | 62777/414113 [00:13<01:14, 4693.66it/s]


 15%|█▌        | 63249/414113 [00:13<01:14, 4698.82it/s]


 15%|█▌        | 63731/414113 [00:13<01:14, 4733.50it/s]


 16%|█▌        | 64205/414113 [00:13<01:14, 4714.90it/s]


 16%|█▌        | 64677/414113 [00:13<01:14, 4668.31it/s]

 41%|████▏     | 171469/414113 [00:36<00:51, 4728.26it/s]


 42%|████▏     | 171944/414113 [00:36<00:51, 4733.14it/s]


 42%|████▏     | 172418/414113 [00:36<00:51, 4709.67it/s]


 42%|████▏     | 172890/414113 [00:37<00:51, 4692.91it/s]


 42%|████▏     | 173360/414113 [00:37<00:51, 4652.79it/s]


 42%|████▏     | 173830/414113 [00:37<00:51, 4664.66it/s]


 42%|████▏     | 174297/414113 [00:37<00:51, 4665.57it/s]


 42%|████▏     | 174777/414113 [00:37<00:50, 4702.86it/s]


 42%|████▏     | 175248/414113 [00:37<00:50, 4689.60it/s]


 42%|████▏     | 175725/414113 [00:37<00:50, 4710.32it/s]


 43%|████▎     | 176197/414113 [00:37<00:50, 4695.33it/s]


 43%|████▎     | 176668/414113 [00:37<00:50, 4697.71it/s]


 43%|████▎     | 177142/414113 [00:37<00:50, 4709.03it/s]


 43%|████▎     | 177613/414113 [00:38<00:50, 4695.60it/s]


 43%|████▎     | 178083/414113 [00:38<00:50, 4693.73it/s]


 43%|████▎     | 178558/414113 [00:38<00:50, 4707.45it/s]


 43%|████▎     | 179029/414113 [00:38<00

 69%|██████▉   | 285228/414113 [01:00<00:27, 4733.04it/s]


 69%|██████▉   | 285706/414113 [01:00<00:27, 4746.31it/s]


 69%|██████▉   | 286181/414113 [01:00<00:27, 4716.44it/s]


 69%|██████▉   | 286656/414113 [01:00<00:26, 4722.75it/s]


 69%|██████▉   | 287129/414113 [01:01<00:27, 4650.86it/s]


 69%|██████▉   | 287595/414113 [01:01<00:27, 4648.36it/s]


 70%|██████▉   | 288087/414113 [01:01<00:26, 4724.61it/s]


 70%|██████▉   | 288568/414113 [01:01<00:26, 4747.56it/s]


 70%|██████▉   | 289057/414113 [01:01<00:26, 4788.26it/s]


 70%|██████▉   | 289562/414113 [01:01<00:25, 4861.88it/s]


 70%|███████   | 290051/414113 [01:01<00:25, 4867.67it/s]


 70%|███████   | 290551/414113 [01:01<00:25, 4906.35it/s]


 70%|███████   | 291042/414113 [01:01<00:25, 4891.78it/s]


 70%|███████   | 291543/414113 [01:01<00:24, 4924.90it/s]


 71%|███████   | 292036/414113 [01:02<00:24, 4920.95it/s]


 71%|███████   | 292533/414113 [01:02<00:24, 4935.49it/s]


 71%|███████   | 293036/414113 [01:02<00

 96%|█████████▌| 398349/414113 [01:24<00:03, 4661.20it/s]


 96%|█████████▋| 398816/414113 [01:24<00:03, 4552.22it/s]


 96%|█████████▋| 399281/414113 [01:25<00:03, 4579.51it/s]


 97%|█████████▋| 399740/414113 [01:25<00:03, 4563.19it/s]


 97%|█████████▋| 400210/414113 [01:25<00:03, 4601.47it/s]


 97%|█████████▋| 400673/414113 [01:25<00:02, 4608.34it/s]


 97%|█████████▋| 401136/414113 [01:25<00:02, 4612.52it/s]


 97%|█████████▋| 401598/414113 [01:25<00:02, 4605.48it/s]


 97%|█████████▋| 402067/414113 [01:25<00:02, 4628.82it/s]


 97%|█████████▋| 402530/414113 [01:25<00:02, 4612.27it/s]


 97%|█████████▋| 403016/414113 [01:25<00:02, 4682.38it/s]


 97%|█████████▋| 403485/414113 [01:25<00:02, 4678.12it/s]


 98%|█████████▊| 403954/414113 [01:26<00:02, 4624.52it/s]


 98%|█████████▊| 404445/414113 [01:26<00:02, 4705.86it/s]


 98%|█████████▊| 404917/414113 [01:26<00:01, 4706.84it/s]


 98%|█████████▊| 405409/414113 [01:26<00:01, 4766.80it/s]


 98%|█████████▊| 405887/414113 [01:26<00

Done (t=0.87s)
creating index...
index created!
Obtaining caption lengths...


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [6]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/2], Step [100/3236], Loss: 3.9335, Perplexity: 51.08626
Epoch [1/2], Step [200/3236], Loss: 3.7916, Perplexity: 44.3258
Epoch [1/2], Step [300/3236], Loss: 3.5009, Perplexity: 33.1462
Epoch [1/2], Step [400/3236], Loss: 3.1457, Perplexity: 23.23555
Epoch [1/2], Step [500/3236], Loss: 3.1121, Perplexity: 22.4675
Epoch [1/2], Step [600/3236], Loss: 3.0488, Perplexity: 21.08959
Epoch [1/2], Step [700/3236], Loss: 3.0701, Perplexity: 21.5446
Epoch [1/2], Step [800/3236], Loss: 2.8808, Perplexity: 17.8294
Epoch [1/2], Step [900/3236], Loss: 3.0763, Perplexity: 21.6773
Epoch [1/2], Step [1000/3236], Loss: 2.9540, Perplexity: 19.1828
Epoch [1/2], Step [1100/3236], Loss: 2.7266, Perplexity: 15.2814
Epoch [1/2], Step [1200/3236], Loss: 2.9294, Perplexity: 18.7171
Epoch [1/2], Step [1300/3236], Loss: 2.7982, Perplexity: 16.4154
Epoch [1/2], Step [1400/3236], Loss: 2.6535, Perplexity: 14.2034
Epoch [1/2], Step [1500/3236], Loss: 2.6829, Perplexity: 14.6274
Epoch [1/2], Step [1600/3236], 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.